### Import necessary libriaries

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\zhaoxi.wang\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.4.1               |           py37_0         544 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         642 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/win-64::altair-2.4.1-py37_0
  branca             conda-forge/noarch::branca-0.3.1-py_0
  folium             conda-forge/noarch::folium-0.5.0-py_0
  vincent            con

### Load the dataframe created in part 2

In [5]:
df_toronto = pd.read_csv('toronto_part2.csv')
df_toronto.head(7)

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
1,1,M3J,North York,"Northwood Park, York University",43.767980,-79.487262
2,2,M4E,East Toronto,The Beaches,43.676357,-79.293031
3,3,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304
4,4,M9C,Etobicoke,Markland Wood,43.643515,-79.577201
5,5,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
6,6,M1S,Scarborough,Agincourt,43.794200,-79.262029


### Created a map of Toronto

In [71]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 11.7)
# add markers to map
for lat, lng, borough, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
map_toronto

### Create a new dataframe with neighbourhoods in Scarborough

In [7]:
CLIENT_ID = 'ELOOIDUGLUPRXECCFQRFY3XSFRBYPXVEBH25K1T1M5301IEK' # your Foursquare ID
CLIENT_SECRET = 'WCZO2OCS4DXTB0YLXLJTBRNKXRUR5ASUFT5O22E0BWX3XRP1' # your Foursquare Secret
VERSION = '20180605'

In [8]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(7)

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
1,3,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304
2,6,M1S,Scarborough,Agincourt,43.794200,-79.262029
3,10,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029
4,30,M1V,Scarborough,"Agincourt North, Milliken",43.815252,-79.284577
5,35,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711
6,39,M1N,Scarborough,Birch Cliff,43.692657,-79.264848


### Create a map for Scarborough and its neighbourhood

In [10]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [74]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

### Get the top 100 venues in the neighbourhood 'Steeles West', from Scarborough

In [75]:
neighbourhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))



Latitude and longitude values of "Highland Creek, Rouge Hill, Port Union" are 43.7845351, -79.16049709999999.


In [76]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [77]:
results = requests.get(url).json()

In [78]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']



In [16]:
venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
3,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
4,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
5,Scarborough Town Centre,Shopping Mall,43.775237,-79.257363
6,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
7,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
8,Jimmy The Greek,Greek Restaurant,43.775112,-79.257119
9,Hot Topic,Clothing Store,43.775450,-79.257929


In [79]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

44 venues were returned by Foursquare.


In [80]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get venues for each neighbourhood in Scarborough

In [81]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Highland Creek, Rouge Hill, Port Union
Dorset Park, Scarborough Town Centre, Wexford Heights
Agincourt
Ionview, Kennedy Park
Agincourt North, Milliken
Morningside, West Hill
Birch Cliff
Tam O'Shanter
Upper Rouge
Woburn
Cliffcrest, Cliffside
Maryvale, Wexford
Scarborough Village
Rouge, Malvern
Clairlea, Golden Mile, Oakridge


In [82]:
scarborough_venues.head(10)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
1,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,Kairali,43.754768,-79.277199,Indian Restaurant
2,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,Kim Kim restaurant,43.753833,-79.276611,Chinese Restaurant
3,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,El Pulgarcito,43.754790,-79.277064,Latin American Restaurant
4,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,Karaikudi Chettinad South Indian Restaurant,43.756042,-79.276276,Indian Restaurant
5,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,Big Al's Pet Supercentre,43.759279,-79.278325,Pet Store
6,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,Pho Vietnam,43.757770,-79.278572,Vietnamese Restaurant
7,Agincourt,43.794200,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
8,Agincourt,43.794200,-79.262029,Subway,43.792823,-79.259681,Sandwich Place
9,Agincourt,43.794200,-79.262029,Twilight,43.791999,-79.258584,Lounge


In [83]:
scarborough_venues.tail(10)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
55,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
56,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Dairy Queen,43.710378,-79.290701,Fast Food Restaurant
57,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Warden Ave & St. Clair Ave E,43.712057,-79.281005,Intersection
58,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,TTC Bus #68 Warden,43.711778,-79.279714,Bus Line
59,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Warden Subway Station,43.711229,-79.279602,Metro Station
60,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,TTC Bus 102 Markham Road,43.711381,-79.279588,Bus Line
61,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Bakery On The Go,43.711271,-79.279506,Bakery
62,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Cafe on the go,43.711151,-79.279469,Bakery
63,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Warden Woods Park,43.710527,-79.278966,Park
64,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Clairlea Futbol Centre,43.715234,-79.286506,Soccer Field


In [85]:
scarborough_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, Milliken",3,3,3,3,3,3
Birch Cliff,4,4,4,4,4,4
"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Cliffcrest, Cliffside",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",6,6,6,6,6,6
"Highland Creek, Rouge Hill, Port Union",1,1,1,1,1,1
"Ionview, Kennedy Park",4,4,4,4,4,4
"Maryvale, Wexford",7,7,7,7,7,7


In [86]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 45 uniques categories.


In [87]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighbourhood'] = scarborough_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighbourhood,American Restaurant,Auto Garage,Bakery,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant
0,"Highland Creek, Rouge Hill, Port Union",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Dorset Park, Scarborough Town Centre, Wexford ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Dorset Park, Scarborough Town Centre, Wexford ...",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Dorset Park, Scarborough Town Centre, Wexford ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Dorset Park, Scarborough Town Centre, Wexford ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [88]:
scarb_grouped = scarb_onehot.groupby('Neighbourhood').mean().reset_index()
scarb_grouped.head(7)

,Neighbourhood,American Restaurant,Auto Garage,Bakery,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.0,0.000000,0.0,0.25,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.25,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.25,0.0,0.25,0.0,0.000000,0.0,0.0,0.000000
1,"Agincourt North, Milliken",0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.00,0.000000,0.333333,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.333333,0.000000,0.0,0.0,0.333333,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000
2,Birch Cliff,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.25,0.000000,0.000000,0.25,0.0,0.0,0.0,0.0,0.000000,0.0,0.25,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.25,0.0,0.000000,0.0,0.0,0.000000
3,"Clairlea, Golden Mile, Oakridge",0.0,0.0,0.222222,0.0,0.00,0.222222,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.111111,0.0,0.00,0.000000,0.0,0.111111,0.0,0.0,0.000000,0.00,0.0,0.111111,0.0,0.0,0.0,0.0,0.111111,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.111111,0.0,0.0,0.000000
4,"Cliffcrest, Cliffside",0.5,0.0,0.000000,0.0,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.5,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000
5,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.00,0.166667,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.333333,0.0,0.000000,0.0,0.0,0.166667,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.166667,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.166667
6,"Highland Creek, Rouge Hill, Port Union",0.0,0.0,0.000000,1.0,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000


### Get top 10 venues per neighbourhood

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [93]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = scarb_grouped['Neighbourhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Breakfast Spot,Sandwich Place,Vietnamese Restaurant,Convenience Store,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
1,"Agincourt North, Milliken",Coffee Shop,Playground,Park,College Stadium,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,Birch Cliff,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,Fried Chicken Joint,Fast Food Restaurant
3,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Soccer Field,Fast Food Restaurant,Metro Station,Intersection,Park,Vietnamese Restaurant,Department Store,Indian Restaurant
4,"Cliffcrest, Cliffside",American Restaurant,Motel,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
5,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Vietnamese Restaurant,Pet Store,Latin American Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
6,"Highland Creek, Rouge Hill, Port Union",Bar,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
7,"Ionview, Kennedy Park",Bus Station,Discount Store,Coffee Shop,Department Store,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint
8,"Maryvale, Wexford",Auto Garage,Bakery,Smoke Shop,Shopping Mall,Sandwich Place,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Convenience Store,General Entertainment
9,"Morningside, West Hill",Intersection,Spa,Breakfast Spot,Electronics Store,Rental Car Location,Medical Center,Pizza Place,Mexican Restaurant,Convenience Store,General Entertainment


### Run k-means to cluster the neighbourhoods into 5 clusters

In [90]:
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 4, 1, 2, 1, 1, 1])

In [91]:
scarb_merged = scarborough_data.drop(14)
scarb_merged.head()

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
1,3,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304
2,6,M1S,Scarborough,Agincourt,43.794200,-79.262029
3,10,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029
4,30,M1V,Scarborough,"Agincourt North, Milliken",43.815252,-79.284577


### Include kmeans.labels_ into the original Scarborough dataframe

In [94]:
# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
scarb_merged = scarb_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

scarb_merged.head() # check the last columns!

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,1,Bar,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
1,3,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,0,Indian Restaurant,Vietnamese Restaurant,Pet Store,Latin American Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,6,M1S,Scarborough,Agincourt,43.794200,-79.262029,1,Lounge,Skating Rink,Breakfast Spot,Sandwich Place,Vietnamese Restaurant,Convenience Store,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
3,10,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029,1,Bus Station,Discount Store,Coffee Shop,Department Store,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint
4,30,M1V,Scarborough,"Agincourt North, Milliken",43.815252,-79.284577,4,Coffee Shop,Playground,Park,College Stadium,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


### Visualize the clusters in the map

In [96]:
# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighbourhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Check the clusters

In [97]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1P,-79.273304,0,Indian Restaurant,Vietnamese Restaurant,Pet Store,Latin American Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
11,M1R,-79.295849,0,Auto Garage,Bakery,Smoke Shop,Shopping Mall,Sandwich Place,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Convenience Store,General Entertainment
13,M1B,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Italian Restaurant,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store


In [98]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1C,-79.160497,1,Bar,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1S,-79.262029,1,Lounge,Skating Rink,Breakfast Spot,Sandwich Place,Vietnamese Restaurant,Convenience Store,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
3,M1K,-79.262029,1,Bus Station,Discount Store,Coffee Shop,Department Store,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint
5,M1E,-79.188711,1,Intersection,Spa,Breakfast Spot,Electronics Store,Rental Car Location,Medical Center,Pizza Place,Mexican Restaurant,Convenience Store,General Entertainment
7,M1T,-79.304302,1,Pizza Place,Chinese Restaurant,Noodle House,Italian Restaurant,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Rental Car Location,Pharmacy,Electronics Store
8,M1X,-79.205636,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,M1G,-79.216917,1,Coffee Shop,Korean Restaurant,Insurance Office,Convenience Store,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
12,M1J,-79.239476,1,Playground,Convenience Store,Vietnamese Restaurant,College Stadium,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [99]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1N,-79.264848,2,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,Fried Chicken Joint,Fast Food Restaurant


In [100]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M1M,-79.239476,3,American Restaurant,Motel,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [101]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M1V,-79.284577,4,Coffee Shop,Playground,Park,College Stadium,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
